In [1]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [2]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [3]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [4]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [5]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [6]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [7]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [8]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '947688', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '250860', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '283412', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '775051', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [9]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [10]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [11]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [12]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [13]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
38627,CN0100100000000,1,1,"Autauga County, AL",Mar-21,"25,433","24,696",737,2.9,01001
38628,CN0100300000000,1,3,"Baldwin County, AL",Mar-21,"96,447","93,539","2,908",3.0,01003
38629,CN0100500000000,1,5,"Barbour County, AL",Mar-21,"8,240","7,776",464,5.6,01005
38630,CN0100700000000,1,7,"Bibb County, AL",Mar-21,"8,526","8,236",290,3.4,01007
38631,CN0100900000000,1,9,"Blount County, AL",Mar-21,"24,650","24,058",592,2.4,01009


In [14]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [15]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41841,CN7214500000000,72,145,"Vega Baja Municipio, PR",Mar-21,"12,962","11,733",1229,9.5,72145
41842,CN7214700000000,72,147,"Vieques Municipio, PR",Mar-21,"2,550","2,273",277,10.9,72147
41843,CN7214900000000,72,149,"Villalba Municipio, PR",Mar-21,"7,021","6,209",812,11.6,72149
41844,CN7215100000000,72,151,"Yabucoa Municipio, PR",Mar-21,"8,386","7,539",847,10.1,72151
41845,CN7215300000000,72,153,"Yauco Municipio, PR",Mar-21,"9,174","8,127",1047,11.4,72153


In [16]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [17]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [18]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [19]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [20]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [21]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [22]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [23]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [24]:
covid.tail(100)

,date,county,state,fips,cases,deaths
1413800,2021-06-12,Webster,West Virginia,54101.0,543,8.0
1413801,2021-06-12,Wetzel,West Virginia,54103.0,1382,31.0
1413802,2021-06-12,Wirt,West Virginia,54105.0,454,4.0
1413803,2021-06-12,Wood,West Virginia,54107.0,7918,176.0
1413804,2021-06-12,Wyoming,West Virginia,54109.0,2033,39.0
1413805,2021-06-12,Adams,Wisconsin,55001.0,1962,19.0
1413806,2021-06-12,Ashland,Wisconsin,55003.0,1396,15.0
1413807,2021-06-12,Barron,Wisconsin,55005.0,6043,84.0
1413808,2021-06-12,Bayfield,Wisconsin,55007.0,1293,20.0
1413809,2021-06-12,Brown,Wisconsin,55009.0,35613,296.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [25]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
463,06/06/2021,125,33,24,4,1,180,237,24,9,9,21,13,7,1,0,40,57,5,1,1,36,3,7,1,0,50,65,8,3,3,20,4,4,1,0,25,32,3,1,1,33,8,4,1,0,47,61,7,3,3,15,4,2,0,0,18,22,1,1,1,11000
464,06/07/2021,206,74,23,6,0,194,252,24,8,9,54,25,6,2,0,46,64,4,1,1,63,17,8,0,0,55,68,8,3,3,27,10,1,0,0,27,34,3,1,1,44,18,7,4,0,48,62,7,3,3,18,4,1,0,0,19,23,2,1,1,11000
465,06/08/2021,164,53,29,9,0,179,234,24,8,8,37,17,10,0,0,42,60,5,1,1,55,11,4,3,0,54,65,7,2,2,20,10,4,1,0,22,31,3,1,1,35,10,9,3,0,43,56,7,3,3,16,4,2,2,0,17,21,2,1,1,11000
466,06/09/2021,192,47,12,5,0,174,226,21,7,7,53,9,7,1,0,43,58,5,1,1,58,13,3,2,0,53,64,6,2,2,30,7,0,0,0,23,31,2,1,1,37,10,1,2,0,40,52,5,2,2,14,8,1,0,0,16,21,2,1,1,11000
467,06/10/2021,130,46,0,3,2,165,217,18,6,6,27,14,0,1,0,41,56,5,1,1,38,10,0,0,0,50,61,5,2,2,23,11,0,1,0,23,32,2,1,1,30,7,0,1,0,36,48,4,2,2,12,2,0,0,0,16,20,1,0,0,11000


In [26]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [27]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [28]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [29]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [30]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [31]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [32]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
463,06/06/2021,21,1,New York,Bronx,36005.0
464,06/07/2021,54,2,New York,Bronx,36005.0
465,06/08/2021,37,0,New York,Bronx,36005.0
466,06/09/2021,53,1,New York,Bronx,36005.0
467,06/10/2021,27,1,New York,Bronx,36005.0


In [33]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [34]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [35]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [36]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
463,06/06/2021,36,1,Kings,New York,36047.0
464,06/07/2021,63,0,Kings,New York,36047.0
465,06/08/2021,55,3,Kings,New York,36047.0
466,06/09/2021,58,2,Kings,New York,36047.0
467,06/10/2021,38,0,Kings,New York,36047.0


In [37]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [38]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [39]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [40]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [41]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,598,33


In [42]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [43]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [44]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
454,05/28/2021,65,3,New York,Queens,36081.0,2021-05-28
455,05/29/2021,40,5,New York,Queens,36081.0,2021-05-29
456,05/30/2021,27,1,New York,Queens,36081.0,2021-05-30
457,05/31/2021,35,4,New York,Queens,36081.0,2021-05-31
458,06/01/2021,69,2,New York,Queens,36081.0,2021-06-01
459,06/02/2021,63,4,New York,Queens,36081.0,2021-06-02
460,06/03/2021,57,2,New York,Queens,36081.0,2021-06-03
461,06/04/2021,44,3,New York,Queens,36081.0,2021-06-04
462,06/05/2021,27,2,New York,Queens,36081.0,2021-06-05
463,06/06/2021,33,1,New York,Queens,36081.0,2021-06-06


In [45]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
418,04/22/2021,Richmond,New York,36085.0,128,3.0,2021-04-22
419,04/23/2021,Richmond,New York,36085.0,134,2.0,2021-04-23
420,04/24/2021,Richmond,New York,36085.0,79,1.0,2021-04-24
421,04/25/2021,Richmond,New York,36085.0,69,2.0,2021-04-25
422,04/26/2021,Richmond,New York,36085.0,121,1.0,2021-04-26
423,04/27/2021,Richmond,New York,36085.0,98,2.0,2021-04-27
424,04/28/2021,Richmond,New York,36085.0,111,0.0,2021-04-28
425,04/29/2021,Richmond,New York,36085.0,90,1.0,2021-04-29
426,04/30/2021,Richmond,New York,36085.0,78,3.0,2021-04-30
427,05/01/2021,Richmond,New York,36085.0,58,2.0,2021-05-01


In [46]:
print (data_date_dt)

2021-06-13 00:00:00


In [47]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
1410654,1001.0,113.0
1410655,1003.0,313.0


In [48]:
Kings_death.head(2)

,fips,deaths
0,36047.0,8628


In [49]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
1413655,51690.0,78.0
1413656,51115.0,12.0
1413657,51117.0,67.0
1413658,51119.0,20.0
1413659,51121.0,96.0
1413660,51125.0,13.0
1413661,51127.0,18.0
1413662,51700.0,235.0
1413663,51710.0,266.0
1413664,51131.0,36.0


In [50]:
covid_death.shape

(3251, 2)

In [51]:
print (County_covid_death)

{1001.0: 113.0, 1003.0: 313.0, 1005.0: 59.0, 1007.0: 64.0, 1009.0: 139.0, 1011.0: 42.0, 1013.0: 71.0, 1015.0: 325.0, 1017.0: 123.0, 1019.0: 45.0, 1021.0: 116.0, 1023.0: 25.0, 1025.0: 61.0, 1027.0: 59.0, 1029.0: 43.0, 1031.0: 126.0, 1033.0: 139.0, 1035.0: 30.0, 1037.0: 29.0, 1039.0: 122.0, 1041.0: 57.0, 1043.0: 200.0, 1045.0: 114.0, 1047.0: 156.0, 1049.0: 189.0, 1051.0: 212.0, 1053.0: 80.0, 1055.0: 361.0, 1057.0: 62.0, 1059.0: 83.0, 1061.0: 81.0, 1063.0: 34.0, 1065.0: 78.0, 1067.0: 43.0, 1069.0: 287.0, 1071.0: 113.0, 1073.0: 1563.0, 1075.0: 37.0, 1077.0: 248.0, 1079.0: 100.0, 1081.0: 175.0, 1083.0: 157.0, 1085.0: 53.0, 1087.0: 51.0, 1089.0: 522.0, 1091.0: 65.0, 1093.0: 106.0, 1095.0: 230.0, 1097.0: 826.0, 1099.0: 41.0, 1101.0: 610.0, 1103.0: 285.0, 1105.0: 26.0, 1107.0: 62.0, 1109.0: 78.0, 1111.0: 44.0, 1113.0: 41.0, 1117.0: 254.0, 1115.0: 250.0, 1119.0: 32.0, 1121.0: 184.0, 1123.0: 154.0, 1125.0: 458.0, 1127.0: 279.0, 1129.0: 39.0, 1131.0: 30.0, 1133.0: 72.0, 2013.0: 3.0, 2016.0: 0.0, 

In [52]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [53]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [54]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
1368763,2021-05-30,New Haven,Connecticut,9009.0,91876,2119.0,2021-05-30,-14 days,NaN
1372009,2021-05-31,New Haven,Connecticut,9009.0,91876,2119.0,2021-05-31,-13 days,0.0
1375255,2021-06-01,New Haven,Connecticut,9009.0,91939,2121.0,2021-06-01,-12 days,63.0
1378502,2021-06-02,New Haven,Connecticut,9009.0,91959,2122.0,2021-06-02,-11 days,20.0
1381748,2021-06-03,New Haven,Connecticut,9009.0,91980,2121.0,2021-06-03,-10 days,21.0
1384995,2021-06-04,New Haven,Connecticut,9009.0,92013,2121.0,2021-06-04,-9 days,33.0
1388241,2021-06-05,New Haven,Connecticut,9009.0,92013,2121.0,2021-06-05,-8 days,0.0
1391487,2021-06-06,New Haven,Connecticut,9009.0,92013,2121.0,2021-06-06,-7 days,0.0
1394733,2021-06-07,New Haven,Connecticut,9009.0,92079,2124.0,2021-06-07,-6 days,66.0
1397981,2021-06-08,New Haven,Connecticut,9009.0,92094,2124.0,2021-06-08,-5 days,15.0


In [55]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [56]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,260.0


In [57]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
418,04/22/2021,Richmond,New York,36085.0,128,3.0,2021-04-22
419,04/23/2021,Richmond,New York,36085.0,134,2.0,2021-04-23
420,04/24/2021,Richmond,New York,36085.0,79,1.0,2021-04-24
421,04/25/2021,Richmond,New York,36085.0,69,2.0,2021-04-25
422,04/26/2021,Richmond,New York,36085.0,121,1.0,2021-04-26
423,04/27/2021,Richmond,New York,36085.0,98,2.0,2021-04-27
424,04/28/2021,Richmond,New York,36085.0,111,0.0,2021-04-28
425,04/29/2021,Richmond,New York,36085.0,90,1.0,2021-04-29
426,04/30/2021,Richmond,New York,36085.0,78,3.0,2021-04-30
427,05/01/2021,Richmond,New York,36085.0,58,2.0,2021-05-01


In [58]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [59]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [60]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [61]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [62]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3219.0,72153.0,6.0,36439,72153,PUERTO RICO
3216,0.0,0.0,0.0,890,2060,ALASKA
3217,0.0,0.0,0.0,2132,2105,ALASKA
3218,0.0,0.0,0.0,689,2282,ALASKA
3219,0.0,0.0,0.0,1375,2164,ALASKA


In [63]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-06-14 00:48:24.033058
6


In [64]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [65]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [66]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [67]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [68]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 113.0
1003.0 313.0
1005.0 59.0
1007.0 64.0
1009.0 139.0
1011.0 42.0
1013.0 71.0
1015.0 325.0
1017.0 123.0
1019.0 45.0
1021.0 116.0
1023.0 25.0
1025.0 61.0
1027.0 59.0
1029.0 43.0
1031.0 126.0
1033.0 139.0
1035.0 30.0
1037.0 29.0
1039.0 122.0
1041.0 57.0
1043.0 200.0
1045.0 114.0
1047.0 156.0
1049.0 189.0
1051.0 212.0
1053.0 80.0
1055.0 361.0
1057.0 62.0
1059.0 83.0
1061.0 81.0
1063.0 34.0
1065.0 78.0
1067.0 43.0
1069.0 287.0
1071.0 113.0
1073.0 1563.0
1075.0 37.0
1077.0 248.0
1079.0 100.0
1081.0 175.0
1083.0 157.0
1085.0 53.0
1087.0 51.0
1089.0 522.0
1091.0 65.0
1093.0 106.0
1095.0 230.0
1097.0 826.0
1099.0 41.0
1101.0 610.0
1103.0 285.0
1105.0 26.0
1107.0 62.0
1109.0 78.0
1111.0 44.0
1113.0 41.0
1117.0 254.0
1115.0 250.0
1119.0 32.0
1121.0 184.0
1123.0 154.0
1125.0 458.0
1127.0 279.0
1129.0 39.0
1131.0 30.0
1133.0 72.0
2013.0 3.0
2016.0 0.0
2020.0 174.0
2050.0 22.0
2997.0 0.0
2068.0 0.0
2070.0 2.0
2090.0 44.0
2100.0 0.0
2110.0 3.0
2122.0 19.0
2130.0 1.0
2150.0 6.0
2158.0 4.0
21

21213.0 42.0
21215.0 25.0
21217.0 52.0
21219.0 28.0
21221.0 15.0
21223.0 7.0
21225.0 16.0
21227.0 181.0
21229.0 41.0
21231.0 50.0
21233.0 21.0
21235.0 61.0
21237.0 6.0
21239.0 21.0
22001.0 195.0
22003.0 92.0
22005.0 167.0
22007.0 36.0
22009.0 123.0
22011.0 77.0
22013.0 82.0
22015.0 308.0
22017.0 759.0
22019.0 447.0
22021.0 29.0
22023.0 7.0
22025.0 38.0
22027.0 55.0
22029.0 61.0
22031.0 78.0
22033.0 829.0
22035.0 27.0
22037.0 112.0
22039.0 94.0
22041.0 107.0
22043.0 58.0
22045.0 159.0
22047.0 100.0
22049.0 51.0
22051.0 901.0
22053.0 97.0
22059.0 37.0
22055.0 291.0
22057.0 205.0
22061.0 93.0
22063.0 209.0
22065.0 40.0
22067.0 78.0
22069.0 93.0
22071.0 799.0
22073.0 457.0
22075.0 27.0
22077.0 75.0
22079.0 322.0
22081.0 38.0
22083.0 56.0
22085.0 62.0
22087.0 54.0
22089.0 97.0
22091.0 12.0
22093.0 50.0
22095.0 141.0
22097.0 261.0
22099.0 118.0
22101.0 141.0
22103.0 528.0
22105.0 302.0
22107.0 8.0
22109.0 225.0
22111.0 81.0
nan 0.0
22113.0 125.0
22115.0 104.0
22117.0 134.0
22119.0 113.0
2212

46073.0 16.0
46075.0 0.0
46077.0 18.0
46079.0 20.0
46081.0 48.0
46083.0 78.0
46085.0 11.0
46091.0 6.0
46087.0 25.0
46089.0 4.0
46093.0 31.0
46095.0 2.0
46097.0 10.0
46099.0 363.0
46101.0 20.0
46102.0 49.0
46103.0 201.0
46105.0 14.0
46107.0 4.0
46109.0 41.0
46111.0 3.0
46115.0 26.0
46117.0 2.0
46119.0 3.0
46121.0 33.0
46123.0 17.0
46125.0 57.0
46127.0 43.0
46129.0 15.0
46135.0 34.0
46137.0 9.0
47001.0 172.0
47003.0 131.0
47005.0 42.0
47007.0 11.0
47009.0 196.0
47011.0 153.0
47013.0 63.0
47015.0 32.0
47017.0 83.0
47019.0 159.0
47021.0 56.0
47023.0 51.0
47025.0 74.0
47027.0 31.0
47029.0 100.0
47031.0 125.0
47033.0 50.0
47035.0 140.0
47037.0 960.0
47041.0 55.0
47039.0 39.0
47043.0 116.0
47045.0 108.0
47047.0 84.0
47049.0 49.0
47051.0 91.0
47053.0 148.0
47055.0 100.0
47057.0 49.0
47059.0 159.0
47061.0 35.0
47063.0 174.0
47065.0 504.0
47067.0 13.0
47069.0 65.0
47071.0 69.0
47073.0 113.0
47075.0 62.0
47077.0 78.0
47079.0 76.0
47081.0 47.0
47083.0 33.0
47085.0 28.0
47087.0 36.0
47089.0 127.0
4

In [69]:
K_c.head()

,fips,cases,deaths
0,36047.0,598,33


In [70]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [71]:
print (K)

{36047.0: 598}


In [72]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 60.0
1001 73.0
1009 61.0
1013 14.0
1015 70.0
1017 51.0
1031 13.0
1033 47.0
1039 46.0
1043 78.0
1045 27.0
1051 67.0
1055 152.0
1067 4.0
1069 34.0
1071 35.0
1077 71.0
1079 45.0
1083 62.0
1089 260.0
1095 105.0
1097 374.0
1103 94.0
1111 22.0
1113 39.0
1115 112.0
1117 153.0
1121 68.0
2261 4.0
4021 554.0
5009 50.0
5011 35.0
5033 44.0
5037 12.0
5045 96.0
5047 7.0
5051 71.0
5053 29.0
5063 52.0
5083 21.0
5085 93.0
5087 7.0
5115 22.0
5117 3.0
5121 18.0
5125 159.0
5131 113.0
5145 73.0
6007 153.0
6017 54.0
6023 130.0
6027 3.0
6061 329.0
6079 90.0
6089 150.0
6093 34.0
8019 7.0
8039 41.0
8047 8.0
8051 20.0
8057 2.0
8065 18.0
8067 44.0
8071 11.0
8075 21.0
8077 652.0
8083 38.0
8085 66.0
8093 23.0
8099 7.0
8101 281.0
8103 4.0
8119 61.0
8121 8.0
9005 40.0
9007 40.0
9011 63.0
9015 32.0
10001 64.0
12019 214.0
12033 126.0
12089 108.0
12101 457.0
12109 202.0
12113 139.0
13015 72.0
13039 34.0
13045 42.0
13047 84.0
13057 117.0
13073 44.0
13077 62.0
13085 9.0
13113 41.0
13115 22.0
13123 42.0
13127 16.0
1

44001 22.0
44005 50.0
44009 48.0
45003 25.0
45007 83.0
45015 64.0
45023 22.0
45035 67.0
45045 218.0
45055 18.0
45057 32.0
45063 59.0
45073 75.0
45077 87.0
45083 138.0
45091 109.0
46015 1.0
46019 5.0
46025 0.0
46029 3.0
46039 0.0
46043 1.0
46045 2.0
46051 0.0
46053 1.0
46055 1.0
46059 1.0
46063 0.0
46067 0.0
46077 1.0
46079 1.0
46081 4.0
46093 8.0
46097 1.0
46101 1.0
46105 0.0
46111 0.0
46115 3.0
46117 1.0
46119 2.0
46123 1.0
46125 1.0
46129 1.0
47001 42.0
47003 152.0
47009 206.0
47011 106.0
47015 6.0
47019 22.0
47021 57.0
47025 31.0
47031 38.0
47033 22.0
47039 19.0
47041 40.0
47043 172.0
47047 22.0
47051 59.0
47053 65.0
47055 17.0
47063 64.0
47065 171.0
47071 16.0
47077 41.0
47079 142.0
47081 23.0
47089 49.0
47099 82.0
47101 32.0
47103 31.0
47107 30.0
47111 33.0
47113 184.0
47115 67.0
47117 87.0
47119 124.0
47127 6.0
47131 40.0
47133 28.0
47139 7.0
47141 430.0
47143 25.0
47145 26.0
47147 94.0
47149 153.0
47155 62.0
47159 26.0
47161 11.0
47163 116.0
47165 144.0
47167 26.0
47177 62.0
471

26049 115.0
26069 12.0
26095 9.0
26129 14.0
26153 4.0
27031 2.0
27053 481.0
27087 1.0
27103 6.0
28005 12.0
28009 4.0
28015 1.0
28023 1.0
28039 13.0
28043 7.0
28071 41.0
28087 44.0
28091 4.0
28107 31.0
28111 1.0
28117 15.0
28145 16.0
28147 2.0
29001 9.0
29015 14.0
29023 18.0
29029 76.0
29063 5.0
29149 8.0
29185 7.0
29197 0.0
29199 0.0
29229 34.0
30023 3.0
30041 27.0
30053 2.0
30061 15.0
30101 1.0
31005 0.0
31133 3.0
31153 41.0
32013 9.0
32017 14.0
32029 3.0
32031 177.0
34003 291.0
35001 325.0
35015 33.0
35017 22.0
35021 0.0
35035 43.0
35037 8.0
35049 73.0
35061 41.0
36001 61.0
36041 0.0
36067 212.0
37041 8.0
37051 368.0
37075 27.0
37119 674.0
37131 9.0
37133 219.0
37147 212.0
38105 50.0
39009 13.0
39035 665.0
39061 345.0
39105 9.0
39121 8.0
39145 25.0
39163 4.0
40005 5.0
40023 1.0
40059 0.0
40067 1.0
40089 17.0
40095 1.0
40141 0.0
41011 12.0
41021 11.0
41023 18.0
41033 52.0
41047 406.0
41055 3.0
41059 116.0
41069 1.0
42003 377.0
42045 219.0
45001 2.0
45013 40.0
45029 13.0
45081 2.0
4508

46017 0.0
46031 3.0
47095 17.0
48061 369.0
48215 753.0
48377 1.0
48445 7.0
48479 2545.0
48489 51.0
53001 23.0
54047 22.0
1085 8.0
2158 10.0
6025 66.0
12049 44.0
12051 37.0
13037 1.0
13265 0.0
22065 5.0
28055 0.0
28123 9.0
28135 4.0
28157 10.0
35031 19.0
36005 115.0
37079 39.0
45069 38.0
46121 2.0
46137 0.0
48047 13.0
48229 1.0
48323 731.0
48427 54.0
28021 1.0
45005 4.0
46102 3.0
48131 3.0
48505 6.0
48507 4.0
48127 125.0
48247 2.0
72009 2.0
72011 2.0
72013 12.0
72017 4.0
72025 23.0
72027 4.0
72039 2.0
72041 4.0
72043 3.0
72054 7.0
72059 2.0
72063 19.0
72065 4.0
72069 14.0
72071 6.0
72085 2.0
72099 7.0
72101 6.0
72115 2.0
72119 13.0
72137 18.0
72145 11.0
72149 2.0
72153 6.0
72003 11.0
72005 6.0
72007 7.0
72021 39.0
72023 8.0
72029 11.0
72031 49.0
72033 7.0
72035 4.0
72037 3.0
72061 19.0
72075 13.0
72077 11.0
72089 8.0
72091 6.0
72103 7.0
72105 4.0
72109 0.0
72111 5.0
72121 9.0
72129 10.0
72131 5.0
72135 20.0
72139 14.0
72141 2.0
72147 0.0
72151 12.0
72051 16.0
72053 10.0
72057 7.0
72067 

In [73]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 197
36061 290
36047 598 598
36081 501
36005 474


In [74]:
print (covid_cases_county_ny_times)

{1001.0: 73.0, 1003.0: 154.0, 1005.0: 13.0, 1007.0: 9.0, 1009.0: 61.0, 1011.0: 10.0, 1013.0: 14.0, 1015.0: 70.0, 1017.0: 51.0, 1019.0: 10.0, 1021.0: 31.0, 1023.0: 4.0, 1025.0: 11.0, 1027.0: 18.0, 1029.0: 21.0, 1031.0: 13.0, 1033.0: 47.0, 1035.0: 11.0, 1037.0: 11.0, 1039.0: 46.0, 1041.0: 12.0, 1043.0: 78.0, 1045.0: 27.0, 1047.0: 7.0, 1049.0: 59.0, 1051.0: 67.0, 1053.0: 34.0, 1055.0: 152.0, 1057.0: 26.0, 1059.0: 33.0, 1061.0: 10.0, 1063.0: 6.0, 1065.0: 13.0, 1067.0: 4.0, 1069.0: 34.0, 1071.0: 35.0, 1073.0: 607.0, 1075.0: 9.0, 1077.0: 71.0, 1079.0: 45.0, 1081.0: 119.0, 1083.0: 62.0, 1085.0: 8.0, 1087.0: 9.0, 1089.0: 260.0, 1091.0: 3.0, 1093.0: 59.0, 1095.0: 105.0, 1097.0: 374.0, 1099.0: 3.0, 1101.0: 196.0, 1103.0: 94.0, 1105.0: 6.0, 1107.0: 9.0, 1109.0: 7.0, 1111.0: 22.0, 1113.0: 39.0, 1115.0: 112.0, 1117.0: 153.0, 1119.0: 1.0, 1121.0: 68.0, 1123.0: 40.0, 1125.0: 105.0, 1127.0: 34.0, 1129.0: 17.0, 1131.0: 1.0, 1133.0: 48.0, 2013.0: 0.0, 2016.0: 6.0, 2020.0: 129.0, 2050.0: 8.0, 2068.0: 2.0

In [75]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [76]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [77]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [78]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1) 
        #print (state_dict [s],s)
    return state_dict



In [79]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [80]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [81]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [82]:
#ACI
ACI_State = total_state(ACI_total)

In [83]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [84]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [85]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [86]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 775051
ALABAMA 947688
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 283412
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1790395
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 250860
PUERTO RICO 1622194


In [87]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

County_covid_death[72049]

nan

In [88]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [89]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 77.0 38921 197.83664345725958
1001 113.0 55869 202.2588555370599
1009 139.0 57826 240.37630131774637
1013 71.0 19448 365.076100370218
1015 325.0 113605 286.0789577923507
1017 123.0 33254 369.88031515005713
1031 126.0 52342 240.724466011998
1033 139.0 55241 251.62469904599845
1039 122.0 37049 329.29363815487596
1043 200.0 83768 238.75465571578644
1045 114.0 49172 231.83925811437402
1051 212.0 81209 261.0548091960251
1055 361.0 102268 352.99409394923146
1067 43.0 17205 249.92734670154027
1069 287.0 105882 271.0564590770858
1071 113.0 51626 218.88195870297915
1077 248.0 92729 267.44599855492885
1079 100.0 32924 303.7298019681691
1083 157.0 98915 158.72213516655714
1089 522.0 372909 139.980531443328
1095 230.0 96774 237.66714200095066
1097 826.0 413210 199.8983567677452
1103 285.0 119679 238.13701651918882
1111 44.0 22722 193.64492562274447
1113 41.0 57961 70.7372198547299
1115 250.0 89512 279.2921619447672
1117 254.0 217702 116.6732505902564
1121 184.0 79978 230.06326739853458
2261 

28095 135.0 35252 382.9569953477817
28109 146.0 55535 262.8972719906365
28121 282.0 155271 181.61794539869004
28129 34.0 15916 213.6215129429505
28159 81.0 17955 451.1278195488722
28161 40.0 12108 330.3600925008259
29003 20.0 17712 112.9177958446251
29007 58.0 25388 228.45438789979517
29011 16.0 11754 136.123872724179
29013 30.0 16172 185.5058125154588
29021 183.0 87364 209.46843093264962
29025 15.0 9020 166.29711751662973
29031 138.0 78871 174.9692535913073
29033 24.0 8679 276.52955409609405
29037 114.0 105780 107.77084515031197
29039 21.0 14349 146.3516621367343
29041 8.0 7426 107.72959870724482
29043 93.0 88595 104.97206388622384
29045 7.0 6797 102.98661174047373
29047 170.0 249948 68.01414694256405
29049 65.0 20387 318.8306273605729
29051 129.0 76745 168.08912632744804
29057 31.0 7561 409.99867742362125
29059 28.0 16878 165.89643322668564
29061 11.0 8278 132.88233872916163
29071 185.0 103967 177.9410774572701
29073 46.0 14706 312.79749762001904
29075 20.0 6571 304.36767615279257
29

38099 27.0 10641 253.73555117000282
39003 239.0 102351 233.51017576770136
39005 97.0 53484 181.36265051230274
39007 179.0 97241 184.07873222200513
39011 87.0 45656 190.55545820921674
39013 174.0 67006 259.67823776975195
39015 61.0 43432 140.4494382022472
39017 603.0 383134 157.3861886441819
39019 48.0 26914 178.3458423125511
39021 58.0 38885 149.15777292014914
39023 306.0 134083 228.2168507566209
39025 252.0 206428 122.0764624953979
39027 66.0 41968 157.26267632481893
39029 230.0 101883 225.74914362553127
39033 110.0 41494 265.0985684677303
39037 129.0 51113 252.38197718779955
39039 98.0 38087 257.30564234515714
39043 164.0 74266 220.82783507930952
39045 204.0 157574 129.46298247172757
39051 71.0 42126 168.5419930684138
39053 50.0 29898 167.2352665730149
39055 150.0 93649 160.1725592371515
39057 246.0 168937 145.61641321912902
39059 53.0 38875 136.33440514469453
39063 131.0 75783 172.8619875169893
39065 65.0 31365 207.23736649131195
39067 38.0 15040 252.6595744680851
39069 67.0 27006 2

1075 37.0 13805 268.01883375588557
1081 175.0 164542 106.35582404492469
1093 106.0 29709 356.79423743646703
1109 78.0 33114 235.54991846348977
1123 154.0 40367 381.499739886541
1125 458.0 209355 218.76716581882448
2068 0.0 2097 0.0
2090 44.0 96849 45.43154808000082
2110 3.0 31974 9.382623381497467
2122 19.0 58708 32.363562035838385
2195 0.0 3266 0.0
2220 1.0 8493 11.77440244907571
2230 0.0 1183 0.0
2275 0.0 2502 0.0
4003 292.0 125922 231.88958243992312
4019 2416.0 1047279 230.69306268912104
4025 519.0 235099 220.75806362426044
5003 35.0 19657 178.05361957572367
5005 104.0 41932 248.02060478870555
5007 422.0 279141 151.1780784621392
5015 46.0 28380 162.0859760394644
5019 41.0 22320 183.69175627240142
5023 74.0 24919 296.9621573899434
5025 25.0 7956 314.228255404726
5029 33.0 20846 158.30375131919791
5031 183.0 110332 165.86303157742088
5043 45.0 18219 246.9948954388276
5055 77.0 45325 169.8841698841699
5059 74.0 33771 219.1229161114566
5061 25.0 13202 189.36524768974397
5071 38.0 26578 

26125 2419.0 1257584 192.3529561444802
26131 21.0 5720 367.13286713286715
26141 34.0 12592 270.012706480305
26161 307.0 367601 83.51446269188604
27001 37.0 15886 232.9094800453229
27007 60.0 47188 127.15097058574214
27009 98.0 40889 239.67326175744086
27011 4.0 4991 80.14425966740133
27013 43.0 67653 63.55963519725659
27019 48.0 105089 45.6755702309471
27021 32.0 29779 107.45827596628497
27027 92.0 64222 143.2530908411448
27037 467.0 429021 108.85248041471164
27039 3.0 20934 14.330753797649757
27057 41.0 21491 190.77753478200177
27073 22.0 6623 332.1757511701646
27075 20.0 10641 187.95226012592803
27077 3.0 3740 80.21390374331551
27081 3.0 5639 53.20092214931725
27083 53.0 25474 208.05527204208212
27095 54.0 26277 205.50291129124327
27099 33.0 40062 82.37232289950576
27101 10.0 8194 122.040517451794
27109 102.0 158293 64.43746722849399
27113 20.0 14119 141.65309157872372
27117 26.0 9126 284.90028490028493
27119 72.0 31364 229.56255579645452
27131 110.0 66972 164.24774532640507
27139 13

53043 10.0 10939 91.41603437242892
53053 663.0 904980 73.26128754226612
53055 0.0 17582 0.0
53061 586.0 822083 71.2823401043447
53063 668.0 522798 127.77401596792643
53071 67.0 60760 110.26991441737985
53073 99.0 229247 43.18486174301081
54001 31.0 16441 188.55300772459097
54007 3.0 13957 21.49459052805044
54011 177.0 91945 192.5063896894883
54017 6.0 8448 71.02272727272728
54019 79.0 42406 186.294392302976
54027 36.0 23175 155.33980582524273
54039 352.0 178124 197.61514450607442
54061 90.0 105612 85.2175889103511
54063 17.0 13275 128.060263653484
54069 87.0 41411 210.0891067590737
54071 11.0 6969 157.8418711436361
54077 46.0 33432 137.59272553242403
54081 92.0 73361 125.40723272583526
54093 10.0 6839 146.22020763269484
55001 19.0 20220 93.96636993076163
55007 20.0 15036 133.01409949454643
55013 25.0 15414 162.1902166861295
55019 65.0 34774 186.92126301259563
55025 335.0 546695 61.2773118466421
55029 30.0 27668 108.42850946942316
55033 33.0 45368 72.7384940927526
55037 13.0 4295 302.67

36001 356.0 305506 116.52798963031823
36041 2.0 4416 45.289855072463766
36067 677.0 460528 147.00517666678246
37041 31.0 13943 222.333787563652
37051 317.0 335509 94.48330745225911
37075 12.0 8441 142.16325079966828
37119 972.0 1110356 87.53949183865355
37131 60.0 19483 307.9607863265411
37133 149.0 197938 75.27609655548707
37147 96.0 180742 53.1143840391276
38105 40.0 37589 106.41411051105376
39009 60.0 65327 91.84563809757068
39035 2204.0 1235072 178.45113483262514
39061 1245.0 817473 152.2986080274211
39105 40.0 22907 174.61911206181514
39121 39.0 14424 270.38269550748754
39145 104.0 75314 138.08853599596358
39163 17.0 13085 129.91975544516623
40005 26.0 13758 188.98095653438
40023 27.0 14672 184.02399127589968
40059 8.0 3688 216.91973969631238
40067 16.0 6002 266.5778073975342
40089 94.0 32832 286.30604288499023
40095 23.0 16931 135.8454905203473
40141 17.0 7250 234.48275862068968
41011 37.0 64487 57.37590522120737
41021 1.0 1912 52.30125523012552
41023 7.0 7199 97.23572718433117
4

29221 52.0 24730 210.27092600080874
29510 525.0 300576 174.6646438837432
31037 21.0 10709 196.09674105892242
32009 0.0 873 0.0
32023 108.0 46523 232.14324097758097
32027 20.0 6725 297.3977695167286
34001 688.0 263670 260.9322258884211
35041 59.0 18500 318.9189189189189
36061 3774.0 1628706 231.717694906263
37005 5.0 11137 44.89539373260303
37007 55.0 24446 224.98568272928085
37065 113.0 51472 219.53683556108174
37093 56.0 55234 101.38682695441214
37107 121.0 55949 216.26838728127404
37163 113.0 63531 177.8659237222773
37181 88.0 44535 197.59739530706187
37187 31.0 11580 267.7029360967185
39075 101.0 43960 229.75432211101003
40091 61.0 19596 311.28801796284955
41045 63.0 30571 206.0776552942331
45025 106.0 45650 232.2015334063527
45027 114.0 33745 337.82782634464365
45049 52.0 19222 270.52335865154515
45065 28.0 9463 295.88925287963644
47029 100.0 36004 277.7469170092212
47129 38.0 21403 177.54520394337243
48055 99.0 43664 226.73140344448518
48105 15.0 3464 433.02540415704385
48115 69.0

5107 43.0 17782 241.81756832752222
6029 1400.0 900202 155.52064980970937
6031 247.0 152940 161.5012423172486
6047 474.0 277680 170.7000864304235
12027 97.0 38001 255.2564406199837
12039 99.0 45660 216.81997371879106
12059 49.0 19617 249.78335117500126
13003 22.0 8165 269.4427434170239
13049 36.0 13392 268.81720430107526
13071 127.0 45600 278.50877192982455
13093 44.0 13390 328.60343539955187
13141 69.0 8457 815.8921603405462
13309 24.0 7855 305.537873965627
16033 0.0 845 0.0
21025 9.0 12630 71.25890736342043
21051 46.0 19901 231.14416360986885
21131 8.0 9877 80.99625392325605
21189 16.0 4415 362.400906002265
21193 66.0 25758 256.2310738411367
22027 55.0 15670 350.98915124441606
22039 94.0 33395 281.47926336277885
28001 85.0 30693 276.93610921056916
28011 133.0 30628 434.24317617866006
28027 83.0 22124 375.1581992406436
28063 28.0 6990 400.57224606580826
28079 74.0 22786 324.7608180461687
28083 125.0 28183 443.5297874605258
28119 16.0 6792 235.5712603062426
28143 27.0 9632 280.315614617

In [90]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(2127.0, 854757, 248.84265352608986)

In [91]:
total_state(Covid_death_capita) 

{'NEW MEXICO': 7500.371581093217,
 'ALABAMA': 18567.09808586187,
 'ALASKA': 817.5008537499745,
 'ARIZONA': 4685.467511994157,
 'ARKANSAS': 17053.70130815352,
 'CALIFORNIA': 6169.727063449925,
 'COLORADO': 8017.086096620717,
 'CONNECTICUT': 1609.1587438622514,
 'DELAWARE': 553.1945190244925,
 'FLORIDA': 13229.584821335615,
 'GEORGIA': 44343.67405252212,
 'IDAHO': 5126.669494047789,
 'ILLINOIS': 23718.225823421886,
 'INDIANA': 19933.692650939647,
 'IOWA': 25087.782864022327,
 'KANSAS': 27376.459467182267,
 'KENTUCKY': 22314.61329475261,
 'LOUISIANA': 17454.8353558238,
 'MAINE': 895.4352713452215,
 'MARYLAND': 3854.8177926879916,
 'MASSACHUSETTS': 3000.2584325506323,
 'MICHIGAN': 16062.118453512241,
 'MINNESOTA': 12953.390425759588,
 'MISSISSIPPI': 24553.535773635784,
 'MISSOURI': 18138.1499313528,
 'MONTANA': 11678.745265648207,
 'NEBRASKA': 14668.768563831602,
 'NEVADA': 2175.803238877923,
 'NEW HAMPSHIRE': 839.5818147931591,
 'NEW JERSEY': 5772.626927129017,
 'NEW YORK': 9297.651512438

In [92]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [93]:
Covid_death_capita[72049]

nan

In [94]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [95]:
print(Medicaid_demand_state)

{'NEW MEXICO': 775051.0, 'ALABAMA': 947688.0000000001, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 283412.0, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON': 1053931

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [96]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [97]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 775051.0 76321890.0 1000000 10155.028917653899
ALABAMA 947688.0000000001 76321890.0 1000000 12416.988101316674
ALASKA 231145.0 76321890.0 1000000 3028.554455346952
ARIZONA 1839932.0 76321890.0 1000000 24107.526687297704
ARKANSAS 830467.0000000003 76321890.0 1000000 10881.111565764428
CALIFORNIA 11847711.0 76321890.0 1000000 155233.45923430356
COLORADO 1337805.0 76321890.0 1000000 17528.45743206831
CONNECTICUT 874974.0 76321890.0 1000000 11464.260122489104
DELAWARE 239009.0 76321890.0 1000000 3131.591736001297
FLORIDA 3892551.999999999 76321890.0 1000000 51001.77681658564
GEORGIA 1928702.999999999 76321890.0 1000000 25270.639917329077
IDAHO 340742.0000000001 76321890.0 1000000 4464.538286460151
ILLINOIS 2987496.0000000005 76321890.0 1000000 39143.37027031171
INDIANA 1602975.9999999995 76321890.0 1000000 21002.834180338035
IOWA 699741.0 76321890.0 1000000 9168.287106097609
KANSAS 401103.0000000001 76321890.0 1000000 5255.41230700655
KENTUCKY 1416013.0000000002 76321890.0 10000

In [98]:
print (Covid_state)

{'NEW MEXICO': 1245.0, 'ALABAMA': 3961.0, 'ALASKA': 439.0, 'ARIZONA': 5611.0, 'ARKANSAS': 2518.0, 'CALIFORNIA': 12490.0, 'COLORADO': 7717.0, 'CONNECTICUT': 1017.0, 'DELAWARE': 500.0, 'FLORIDA': 20525.0, 'GEORGIA': 5454.0, 'IDAHO': 1519.0, 'ILLINOIS': 5764.0, 'INDIANA': 5421.0, 'IOWA': 1241.0, 'KANSAS': 1575.0, 'KENTUCKY': 3723.0, 'LOUISIANA': 5113.0, 'MAINE': 802.0, 'MARYLAND': 1427.0, 'MASSACHUSETTS': 1686.0, 'MICHIGAN': 5047.0, 'MINNESOTA': 2269.0, 'MISSISSIPPI': 1712.0, 'MISSOURI': 5792.0, 'MONTANA': 966.0, 'NEBRASKA': 478.0, 'NEVADA': 3400.0, 'NEW HAMPSHIRE': 494.0, 'NEW JERSEY': 3270.0, 'NEW YORK': 5880.0, 'NORTH CAROLINA': 6733.0, 'NORTH DAKOTA': 495.0, 'OHIO': 4923.0, 'OKLAHOMA': 1607.0, 'OREGON': 3648.0, 'PENNSYLVANIA': 6448.0, 'RHODE ISLAND': 638.0, 'SOUTH CAROLINA': 1939.0, 'SOUTH DAKOTA': 201.0, 'TENNESSEE': 6313.0, 'TEXAS': 24417.0, 'UTAH': 3361.0, 'VERMONT': 116.0, 'VIRGINIA': 2731.0, 'WASHINGTON': 8519.0, 'WEST VIRGINIA': 1713.0, 'WISCONSIN': 2161.0, 'WYOMING': 978.0, 'HA

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [99]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



In [100]:
cartesian_pro_county_state

<gurobi.tuplelist (3220 tuples, 2 values each):
 ( 35039 , NEW MEXICO           )
 ( 1001  , ALABAMA              )
 ( 1009  , ALABAMA              )
 ( 1013  , ALABAMA              )
 ( 1015  , ALABAMA              )
 ( 1017  , ALABAMA              )
 ( 1031  , ALABAMA              )
 ( 1033  , ALABAMA              )
 ( 1039  , ALABAMA              )
 ( 1043  , ALABAMA              )
 ( 1045  , ALABAMA              )
 ( 1051  , ALABAMA              )
 ( 1055  , ALABAMA              )
 ( 1067  , ALABAMA              )
 ( 1069  , ALABAMA              )
 ( 1071  , ALABAMA              )
 ( 1077  , ALABAMA              )
 ( 1079  , ALABAMA              )
 ( 1083  , ALABAMA              )
 ( 1089  , ALABAMA              )
 ( 1095  , ALABAMA              )
 ( 1097  , ALABAMA              )
 ( 1103  , ALABAMA              )
 ( 1111  , ALABAMA              )
 ( 1113  , ALABAMA              )
 ( 1115  , ALABAMA              )
 ( 1117  , ALABAMA              )
 ( 1121  , ALABAMA              )


In [101]:
Total_covid_death_cap


{'NEW MEXICO': 7500.371581093217,
 'ALABAMA': 18567.09808586187,
 'ALASKA': 817.5008537499745,
 'ARIZONA': 4685.467511994157,
 'ARKANSAS': 17053.70130815352,
 'CALIFORNIA': 6169.727063449925,
 'COLORADO': 8017.086096620717,
 'CONNECTICUT': 1609.1587438622514,
 'DELAWARE': 553.1945190244925,
 'FLORIDA': 13229.584821335615,
 'GEORGIA': 44343.67405252212,
 'IDAHO': 5126.669494047789,
 'ILLINOIS': 23718.225823421886,
 'INDIANA': 19933.692650939647,
 'IOWA': 25087.782864022327,
 'KANSAS': 27376.459467182267,
 'KENTUCKY': 22314.61329475261,
 'LOUISIANA': 17454.8353558238,
 'MAINE': 895.4352713452215,
 'MARYLAND': 3854.8177926879916,
 'MASSACHUSETTS': 3000.2584325506323,
 'MICHIGAN': 16062.118453512241,
 'MINNESOTA': 12953.390425759588,
 'MISSISSIPPI': 24553.535773635784,
 'MISSOURI': 18138.1499313528,
 'MONTANA': 11678.745265648207,
 'NEBRASKA': 14668.768563831602,
 'NEVADA': 2175.803238877923,
 'NEW HAMPSHIRE': 839.5818147931591,
 'NEW JERSEY': 5772.626927129017,
 'NEW YORK': 9297.651512438

# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [102]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [103]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [104]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [105]:
#print (Medicaid_demand)

In [106]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [107]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)
Covid_capita = order_k(Covid_capita)#{m: Unemployment_capita[m] for m in location}
Covid_death_capita = order_k(Covid_death_capita)

In [108]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [109]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)